# Calculating the weights for for a portfolio of Cash Settled Swaptions to replicate a CMS rate

### Setup
-  Choose the number of swaptions for each Call & Put portfilio. e.g. use 30
-  determine which strikes to use
-  create functions to build the list of strikes & forwards
-  crate a PVBP function



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from financepy.utils import *
from financepy.models.sabr_shifted import *
import scipy.stats as si

In [ ]:

def B76(ff,kk,tt,vol,ss,cp):# ff= forward, kk = strike, ss=displacement
    
    # create displaced forwards & srtikes
    ff=ff+ss
    kk=kk+ss

    d1=(np.log(ff/kk) + 0.5* tt * vol**2)/(vol * (tt**0.5))
    d2=(np.log(ff/kk) - 0.5* tt * vol**2)/(vol * (tt**0.5))

    c=ff*N(d1)-kk*N(d2)
    p=kk*N(-d2)-ff*N(-d1)

    if cp==1:
        retval=c
    else:
        retval=p

    return retval

In [15]:

def getStrikes(anchorStrike, displacement=0, CallPut = "CALL", nCashSettledSwaptions=30):

    if CallPut == "CALL":
        finalStrike = 10 * (anchorStrike + displacement)
    else:
        finalStrike = -displacement + 0.000001

    increment = (finalStrike-anchorStrike)/(nCashSettledSwaptions-1)
    cashSettledSwaptionStrikes = [anchorStrike + i * increment for i in range(nCashSettledSwaptions)]
            
    return cashSettledSwaptionStrikes




def getForwards(anchorStrike, displacement=0, CallPut = "CALL", nCashSettledSwaptions=30):
    
    K=getStrikes(anchorStrike, displacement, CallPut, nCashSettledSwaptions)
    
    increment=K[1]-K[0]

    cashSettledSwaptionForwards =[K[i] + increment/2  for i in range(nCashSettledSwaptions)]
            
    return cashSettledSwaptionForwards


    
def PVBP(swapParRate,swapTenorInMonths,swapFixedLegPaymentTenorInMonths=12):
    

            factor=swapFixedLegPaymentTenorInMonths/12
            nPaymemnts=int(swapTenorInMonths/swapFixedLegPaymentTenorInMonths)
            df = 1/(1 + swapParRate*factor)
            
            s=0.0
            p=1.0
            for i in range(nPaymemnts):
                p*=df
                s+=p
                
            return s * factor


#print(PVBP(0.05,240,6))

def largest(a,b):
    if a>b:
        r=a
    else:
        r=b
    return r

#'######################################################################################

#The weights can be calculated recursively by calculating theterninal payoff

def calculateWeights(callputN,anchorStrike,displacement,swapTenorinMonths,swapFixedLegMonths,nCashSettledSwaptions=30):
    
    
    CSSweights=[] # create the empty list to hold the weights
    swapTenor=swapTenorinMonths

    if callputN==1:
        CallPut="CALL"
    else:
        CallPut="PUT"
    
    
    Strikes=getStrikes(anchorStrike, displacement, CallPut, nCashSettledSwaptions)
    Forwards=getForwards(anchorStrike, displacement, CallPut, nCashSettledSwaptions)

    # the target rate is the CMS rate at each forward rate
    Targets=[Forwards[i]-anchorStrike for i in range(nCashSettledSwaptions)] 


#loop through each forward rate and calculate the weight of the marginal swaption 
#
##                                      $$ 
#                                     $$$
#                                    $$$$
#                                   $$ 
#                                  $$$
#                                 $$$$
##################################################################################
#                                 K1  F1   K2

    for i in range(nCashSettledSwaptions):
        #find weight

        currentFwd=Forwards[i]
        currentPVBP=PVBP(currentFwd,swapTenor,swapFixedLegMonths)

        nFixedStrikes=len(CSSweights) # loopp through the strikes that have already had their weight set
        #print(nFixedStrikes)
        prevPV=0

        if nFixedStrikes>0: # use this once we have calculated the first swaption weight
             for w in range(nFixedStrikes):
                prevPV=prevPV + largest(callputN*(currentFwd-Strikes[w]),0) * currentPVBP * CSSweights[w]
            
        else:
            prevPV=0
           
        requiredPV=Targets[i]-prevPV
        currentWeight= requiredPV / (largest(callputN*(currentFwd-Strikes[i]),0)*currentPVBP)
        CSSweights.append(currentWeight)

    return CSSweights,Strikes



def N(x):
 PolanitzerNormsdist = si.norm.cdf(x,0.0,1.0)
 return (PolanitzerNormsdist)





F=0.03
D=0.0
T=2.0
swapLengthinMonths=240
nCashSettledSwaptions=30

wCalls,kCalls=calculateWeights(1,F,D,swapLengthinMonths,6,nCashSettledSwaptions)
wPuts,kPuts=calculateWeights(-1,F,D,swapLengthinMonths,6,nCashSettledSwaptions)



alpha = 0.20; beta = 0.5; rho = -0.90; nu = 0.817; shift = D

model = SABRShifted(alpha, beta, rho, nu, D)

ann=PVBP(F,swapLengthinMonths,6)
volsSABR = model.black_vol(F, np.array(kCalls), T) 
import pandas as pd
portfolioC=0
for o in range (nCashSettledSwaptions):
    kk=kCalls[o]
    w=wCalls[o]
    v=volsSABR[o]

    portfolioC+=w*ann*B76(F,kk,T,v,D,1)



volsSABR = model.black_vol(F, np.array(kPuts), T) 
portfolioP=0
for o in range (nCashSettledSwaptions):
    kk=kPuts[o]
    w=wPuts[o]
    v=volsSABR[o]

    portfolioP+=w*ann*B76(F,kk,T,v,D,-1)
    #print(portfolioP)

#dd=pd.DataFrame(kCalls#),wCalls,kPuts,wPuts)
s1= "convexity adjustment = " +  str(round((portfolioC+portfolioP)*100,3)) + "%"
s2= "convexity adjusted rate = " + str(round((F+portfolioC+portfolioP)*100,3)) + "%"
print(s1)
print(s2)

convexity adjustment = 0.989%
convexity adjusted rate = 3.989%
